In [35]:
from pyxdf import match_streaminfos, resolve_streams
from mnelab.io.xdf import read_raw_xdf
import mne
import mnelab
import numpy as np
import matplotlib.pyplot as plt
import os.path
import pandas as pd
import scipy
import pickle

In [36]:
reject_criteria = dict(eeg=100e-6)
def read_file(raw):
    raw.filter(2, 20)
    raw2 =raw.copy().resample(150)
    # ica = mne.preprocessing.ICA(n_components = 4, random_state = 0)
    # ica.fit(raw2)
    # raw2 = ica.apply(raw2,exclude = [0])
    events, _ = mne.events_from_annotations(raw2)
    # "Non distraction + congruent":6, "distraction + congruent" :4 , "Non distraction + incongruent":7, "distraction + incongruent" :5
    event_ids = {"Non distraction + incongruent":7, "distraction + incongruent" :5}
    epochs = mne.Epochs(raw2, events,event_id = event_ids,  preload=True,tmin = -0.5, tmax=1 , baseline=(None, 0),  reject=reject_criteria)
    labels=epochs.events[:,-1]
    features=epochs.get_data()
    labels[labels == 5] = 1 # distracted
    #labels[labels == 5] = 1 # distracted
    labels[labels == 7 ] = 0 # nd
    #labels[labels == 7] = 0 # nd

    xx= np.array(features)
        
    NormalizedAll = xx - xx.min();
    NormalizedAll = NormalizedAll / xx.max();
    NormalizedAll=NormalizedAll.reshape(features.shape[0], 4, features.shape[2])

    mat = np.zeros((features.shape[0],4,4))
    for i in range (0,features.shape[0]):
        Pearson_matrix = np.corrcoef(NormalizedAll[i,:,:]);
        abs_Pearson_matrix =  abs(Pearson_matrix)
        mat[i,:,:] = abs_Pearson_matrix -  np.eye(4, 4);

    # epoch_incongruent_ND = mne.Epochs(raw2, events,event_id = event_ids["Non distraction + congruent"],  preload=True,tmin = -0.5, tmax=0.8 , baseline=(None, 0))
    # epoch_incongruent_D = mne.Epochs(raw2, events,event_id = event_ids["distraction + congruent"],  preload=True,tmin = -0.5, tmax=0.8, baseline=(None, 0))

    # epoch_incongruent_ND = mne.Epochs(raw2, events,event_id = event_ids["Non distraction + incongruent"],  preload=True,tmin = -0.5, tmax=0.5 , baseline=(None, 0))
    # epoch_incongruent_D = mne.Epochs(raw2, events,event_id = event_ids["distraction + incongruent"],  preload=True,tmin = -0.5, tmax=0.5, baseline=(None, 0))

    # epoch_D = mne.Epochs(raw2, events,event_id = event_ids["distraction + congruent", "distraction + incongruent"],  preload=True,tmin = -0.5, tmax=0.5 , baseline=(None, 0))
    # epoch_ND = mne.Epochs(raw2, events,event_id = event_ids["Non distraction + congruent", "distraction + incongruent"],  preload=True,tmin = -0.5, tmax=0.5 , baseline=(None, 0))
    
    return mat,labels

Event ids are, 41 =  D + congruent, 42 =  D + incongruent,  51 =  ND + congruent, 52 =  ND + incongruent

In [37]:
mne.set_log_level(verbose='WARNING', return_old_level=False, add_frames=None)

In [38]:
graphs, labels, groups=[],[],[]
for pnum in range(1,21):
    paraticipant = "P"+ str(pnum)
    print(pnum) 

    path ='./'+ paraticipant +'/'
    raw = None 
    for num in range(1,5):
        fileNamei = paraticipant +'_b'  + str(num) + ".xdf"
        path_addr = path + fileNamei
        if (os.path.exists(path_addr)):
            streams = resolve_streams(path_addr)
            stream_id = match_streaminfos(streams, [{"type": "EEG"}])[0]
            rawi = read_raw_xdf(path_addr, stream_ids=[stream_id]).resample(250)
            
            if (raw is not None):
                mne.io.concatenate_raws([raw, rawi], on_mismatch="ignore")
            else:
                raw = rawi  

            if (pnum == 13) or (pnum == 19):
                raw = None      

    if (raw is not None):
        graphs_sess1,label_sess1= read_file(raw)
        graphs.append(graphs_sess1)
        labels.append(label_sess1)
        groups.append([pnum]*(len(label_sess1)))

Stream 2: Calculated effective sampling rate 473.7025 Hz is different from specified rate 256.0000 Hz.


1


Stream 2: Calculated effective sampling rate 455.8237 Hz is different from specified rate 256.0000 Hz.
Stream 2: Calculated effective sampling rate 475.2557 Hz is different from specified rate 256.0000 Hz.
Stream 2: Calculated effective sampling rate 459.6645 Hz is different from specified rate 256.0000 Hz.
Stream 2: Calculated effective sampling rate 476.0124 Hz is different from specified rate 256.0000 Hz.


2


Stream 2: Calculated effective sampling rate 390.8616 Hz is different from specified rate 256.0000 Hz.
/var/folders/x4/6xjb_tfs6vdg6llm0_l4vjz00000gn/T/ipykernel_34696/1335903286.py:17: RuntimeWarning: Omitted 8 annotation(s) that were outside data range.
  mne.io.concatenate_raws([raw, rawi], on_mismatch="ignore")
Stream 2: Calculated effective sampling rate 454.4369 Hz is different from specified rate 256.0000 Hz.
Stream 2: Calculated effective sampling rate 464.3009 Hz is different from specified rate 256.0000 Hz.
Stream 2: Calculated effective sampling rate 467.3554 Hz is different from specified rate 256.0000 Hz.


...

Stream 2: Calculated effective sampling rate 482.7663 Hz is different from specified rate 256.0000 Hz.
Stream 2: Calculated effective sampling rate 475.3240 Hz is different from specified rate 256.0000 Hz.
Stream 2: Calculated effective sampling rate 456.3403 Hz is different from specified rate 256.0000 Hz.
Stream 2: Calculated effective sampling rate 469.7415 Hz is different from specified rate 256.0000 Hz.


19


Stream 2: Calculated effective sampling rate 467.5118 Hz is different from specified rate 256.0000 Hz.
Stream 2: Calculated effective sampling rate 459.9489 Hz is different from specified rate 256.0000 Hz.
Stream 2: Calculated effective sampling rate 470.8543 Hz is different from specified rate 256.0000 Hz.
Stream 2: Calculated effective sampling rate 485.2200 Hz is different from specified rate 256.0000 Hz.


20


Stream 2: Calculated effective sampling rate 471.5687 Hz is different from specified rate 256.0000 Hz.
Stream 2: Calculated effective sampling rate 439.5069 Hz is different from specified rate 256.0000 Hz.
/var/folders/x4/6xjb_tfs6vdg6llm0_l4vjz00000gn/T/ipykernel_34696/1335903286.py:17: RuntimeWarning: Omitted 5 annotation(s) that were outside data range.
  mne.io.concatenate_raws([raw, rawi], on_mismatch="ignore")
Stream 2: Calculated effective sampling rate 475.3133 Hz is different from specified rate 256.0000 Hz.


In [15]:
# import numpy as np
# data_array=np.concatenate(features)
# labels=np.concatenate(labels)
# groups=np.concatenate(groups)

# features.shape,labels.shape,groups.shape

In [39]:
graph_array=np.vstack(graphs)
label_array=np.hstack(labels)
group_array=np.hstack(groups)
print(graph_array.shape,label_array.shape,group_array.shape)

(878, 4, 4) (878,) (878,)


In [40]:
unique, counts = np.unique(group_array, return_counts=True)
print(np.asarray((unique, counts)).T)

selected_graphs = []
selected_labels = []
selected_groups = []

num_of_graphs= 39

for p in unique:
    graph_indexes = np.where(group_array == p)[0]
    no_of_graphs_for_p = len(graph_indexes)
    if no_of_graphs_for_p >= num_of_graphs:
        # only selects the first 16 indexes
        selected_graph_indexes = graph_indexes[:num_of_graphs]
        # selected_graphs = np.append(selected_graphs, graph_array[selected_graph_indexes,:,:])
        # selected_labels = np.append(selected_labels, label_array[selected_graph_indexes])
        # selected_groups = np.append(selected_groups, group_array[selected_graph_indexes])

        selected_graphs.append(graph_array[selected_graph_indexes])
        selected_labels.extend(label_array[selected_graph_indexes])
        selected_groups.extend(group_array[selected_graph_indexes])

selected_graphs = np.concatenate(selected_graphs, axis=0) if selected_graphs else np.array([])
# selected_labels = np.concatenate(selected_labels) if selected_labels else np.array([])
# selected_groups = np.concatenate(selected_groups) if selected_groups else np.array([])        



unique, counts = np.unique(selected_groups, return_counts=True)
print(np.asarray((unique, counts)).T)

[[ 1 39]
 [ 2 69]
 [ 3 56]
 [ 4 88]
 [ 5 55]
 [ 6 49]
 [ 7 56]
 [ 8 65]
 [ 9 62]
 [10 60]
 [11 69]
 [12 16]
 [14 46]
 [15 39]
 [16  2]
 [17 45]
 [18 20]
 [20 42]]
[[ 1 39]
 [ 2 39]
 [ 3 39]
 [ 4 39]
 [ 5 39]
 [ 6 39]
 [ 7 39]
 [ 8 39]
 [ 9 39]
 [10 39]
 [11 39]
 [14 39]
 [15 39]
 [17 39]
 [20 39]]


In [43]:
unique, counts = np.unique(selected_groups, return_counts=True)
print(np.asarray((unique, counts)).T)

[[ 1 39]
 [ 2 39]
 [ 3 39]
 [ 4 39]
 [ 5 39]
 [ 6 39]
 [ 7 39]
 [ 8 39]
 [ 9 39]
 [10 39]
 [11 39]
 [14 39]
 [15 39]
 [17 39]
 [20 39]]


In [42]:
unique, counts = np.unique(selected_labels, return_counts=True)
print(np.asarray((unique, counts)).T)

[[  0 341]
 [  1 244]]


In [32]:
with open('incongruent_graphs.pickle', 'wb') as output:
    pickle.dump(selected_graphs, output)


with open('incongruent_labels.pickle', 'wb') as output:
    pickle.dump(selected_labels, output)


with open('incongruent_groups.pickle', 'wb') as output:
    pickle.dump(selected_groups, output)    
